#Connect Google colab with your Google drive

In [60]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials


In [61]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


# Download required files in colab

In [62]:
downloaded = drive.CreateFile({'id':'15Uql3w-cOK444GraLi50lF2vpSpYseZ9'}) # replace the id with id of file you want to access
downloaded.GetContentFile('model-022-0.791082.h5') #Give the exact name of your file

#Checking the model and the dataset

In [63]:
import keras

new_model=keras.models.load_model('model-022-0.791082.h5')
new_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [64]:
new_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 25, 300)           33529800  
_________________________________________________________________
lstm (LSTM)                  (None, 64)                93440     
_________________________________________________________________
dense (Dense)                (None, 5)                 325       
Total params: 33,623,565
Trainable params: 93,765
Non-trainable params: 33,529,800
_________________________________________________________________


In [65]:
import pandas as pd
import gensim
import numpy as np
import os
import re
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, GRU, LSTM
from keras.layers.embeddings import Embedding
from keras.initializers import Constant
from keras.callbacks import ModelCheckpoint
from keras.models import load_model

In [66]:
downloaded = drive.CreateFile({'id':'1fhypxKeRNfcoZWfb6AOQnaUzb4wopE_6'}) # replace the id with id of file you want to access
downloaded.GetContentFile('Amazon_Unlocked_Mobile.csv')
data=pd.read_csv("Amazon_Unlocked_Mobile.csv")
data.head()

,Product Name,Brand Name,Price,Rating,Reviews,Review Votes
0,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,5,I feel so LUCKY to have found this used (phone...,1.0
1,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,"nice phone, nice up grade from my pantach revu...",0.0
2,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,5,Very pleased,0.0
3,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,It works good but it goes slow sometimes but i...,0.0
4,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,Great phone to replace my lost phone. The only...,0.0


In [67]:
data['Rating']=data['Rating']-1
data.head()

,Product Name,Brand Name,Price,Rating,Reviews,Review Votes
0,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,I feel so LUCKY to have found this used (phone...,1.0
1,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,3,"nice phone, nice up grade from my pantach revu...",0.0
2,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,Very pleased,0.0
3,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,3,It works good but it goes slow sometimes but i...,0.0
4,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,3,Great phone to replace my lost phone. The only...,0.0


In [68]:
data["Rating"].value_counts()

4    223605
0     72350
3     61392
2     31765
1     24728
Name: Rating, dtype: int64

In [69]:
data.drop(['Product Name','Brand Name','Price','Review Votes'],axis=1,inplace=True)

In [70]:
data.head()

,Rating,Reviews
0,4,I feel so LUCKY to have found this used (phone...
1,3,"nice phone, nice up grade from my pantach revu..."
2,4,Very pleased
3,3,It works good but it goes slow sometimes but i...
4,3,Great phone to replace my lost phone. The only...


In [71]:
data.isnull().sum()

Rating      0
Reviews    62
dtype: int64

In [72]:
data = data.dropna(how='any',axis=0)
data.isnull().sum()

Rating     0
Reviews    0
dtype: int64

In [73]:
data["Rating"].value_counts()

4    223580
0     72337
3     61374
2     31763
1     24724
Name: Rating, dtype: int64

In [74]:
data.count()

Rating     413778
Reviews    413778
dtype: int64

#Preprocessing of reviews

In [16]:
def clean_text(text):
    text = text.lower()
    
    pattern = re.compile('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    text = pattern.sub('', text)
    text = " ".join(filter(lambda x:x[0]!='@', text.split()))
    emoji = re.compile("["
                           u"\U0001F600-\U0001FFFF"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    
    text = emoji.sub(r'', text)
    text = text.lower()
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"that's", "that is", text)        
    text = re.sub(r"what's", "what is", text)
    text = re.sub(r"where's", "where is", text) 
    text = re.sub(r"\'ll", " will", text)  
    text = re.sub(r"\'ve", " have", text)  
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"don't", "do not", text)
    text = re.sub(r"did't", "did not", text)
    text = re.sub(r"can't", "can not", text)
    text = re.sub(r"it's", "it is", text)
    text = re.sub(r"couldn't", "could not", text)
    text = re.sub(r"have't", "have not", text)
    text = re.sub(r"[,.\"\'!@#$%^&*(){}?/;`~:<>+=-]", "", text)
    return text

In [17]:
import nltk

In [18]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [19]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [20]:
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

def CleanTokenize(data):
    req_lines = list()
    lines = data["Reviews"].values.tolist()

    for line in lines:
        line = clean_text(line)
        # tokenize the text
        tokens = word_tokenize(line)
        tokens = [w.lower() for w in tokens]
        # remove puntuations
        table = str.maketrans('', '', string.punctuation)
        stripped = [w.translate(table) for w in tokens]
        # remove non alphabetic characters
        words = [word for word in stripped if word.isalpha()]
        stop_words = set(stopwords.words("english"))
        # remove stop words
        stop_words.discard('not')
        words = [w for w in words if not w in stop_words]
        req_lines.append(words)    
    return req_lines

req_lines = CleanTokenize(data)
req_lines[0:5]

[['feel',
  'lucky',
  'found',
  'used',
  'phone',
  'us',
  'not',
  'used',
  'hard',
  'phone',
  'line',
  'someone',
  'upgraded',
  'sold',
  'one',
  'son',
  'liked',
  'old',
  'one',
  'finally',
  'fell',
  'apart',
  'years',
  'didnt',
  'want',
  'upgrade',
  'thank',
  'seller',
  'really',
  'appreciate',
  'honesty',
  'said',
  'used',
  'phonei',
  'recommend',
  'seller',
  'highly',
  'would'],
 ['nice',
  'phone',
  'nice',
  'grade',
  'pantach',
  'revue',
  'clean',
  'set',
  'easy',
  'set',
  'never',
  'android',
  'phone',
  'fantastic',
  'say',
  'least',
  'perfect',
  'size',
  'surfing',
  'social',
  'media',
  'great',
  'phone',
  'samsung'],
 ['pleased'],
 ['works', 'good', 'goes', 'slow', 'sometimes', 'good', 'phone', 'love'],
 ['great',
  'phone',
  'replace',
  'lost',
  'phone',
  'thing',
  'volume',
  'button',
  'not',
  'work',
  'still',
  'go',
  'settings',
  'adjust',
  'job',
  'eligible',
  'upgrade',
  'phone',
  'againthaanks']]

In [21]:
max_length = 25

tokenizer_obj = Tokenizer()
#print(tokenizer_obj.texts_to_sequences())
tokenizer_obj.fit_on_texts(req_lines)
sequences = tokenizer_obj.texts_to_sequences(req_lines)

#Predicted rating based on text data

In [47]:
def predict_rating(s):
    x_final = pd.DataFrame({"Reviews":[s]})
    test_lines = CleanTokenize(x_final)
    test_sequences = tokenizer_obj.texts_to_sequences(test_lines)
    test_review_pad = pad_sequences(test_sequences, maxlen=max_length, padding='post')
    pred = new_model.predict(test_review_pad)
    n=max(pred[0])
    for i in range(5):
        if pred[0][i]==n:
            return i+1

4


#Audio to text conversion and rating prediction

In [23]:
pip install SpeechRecognition

     |████████████████████████████████| 32.8MB 113kB/s 


In [56]:
downloaded = drive.CreateFile({'id':'1I18s7QBx5wZ2Mvl6ZALAk6O__E2PAU4e'}) # replace the id with id of file you want to access
downloaded.GetContentFile('Recording_6.wav')

In [57]:
import speech_recognition as sr
s=input('Enter the file name: ')
filename = s

# initialize the recognizer
r = sr.Recognizer()

# open the file
with sr.AudioFile(filename) as source:
    # listen for the data (load audio to memory)
    audio_data = r.record(source)
    # recognize (convert from speech to text)
    text = r.recognize_google(audio_data)
    print('your review: '+text)
    
s=text
print('suggested rating: '+str(predict_rating(s)))

Enter the file name: Recording_6.wav
your review: the charger is fast but the phone also drains fast
suggested rating: 4
